# Learn Agent Enhanced - LangChain Agent 新手学习版

## 🎯 基于国产DeepSeek的Agent实战教程


In [4]:
# 🔧 第1课：创建你的第一个智能Agent

# 环境配置
import os
from dotenv import load_dotenv
load_dotenv()
print("✅ 环境配置完成")

✅ 环境配置完成


In [8]:
# 步骤1A: 加载本地文档(可选)
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("📖 加载本地文档...")

#方法1: 加载单个文件
loader = TextLoader("data/绍宋.txt", encoding='utf-8')
docs = loader.load()

# # 方法2: 加载整个目录的文档
# loader = DirectoryLoader("data/", glob="**/*.txt", loader_cls=TextLoader)
# docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

print(f"✅ 本地文档加载完成，共{len(documents)}个片段")
print(f"📝 第一个片段预览: {documents[0].page_content[:100] if documents else '无文档'}...")

📖 加载本地文档...
✅ 本地文档加载完成，共151个片段
📝 第一个片段预览: 绍宋
作者：榴弹怕水

内容简介：
　　绍者，一曰继；二曰导。
　　公元1127年，北宋灭亡。旋即，皇九子赵构在万众期待中于商丘登基，继承宋统，改元建炎。
　　然而，三个月内，李纲罢相，陈东被杀，岳飞...


In [14]:
# 步骤1C: 创建本地文档Web服务（可选）
from flask import Flask, send_from_directory, abort
import os
import threading
import time
import socket

# 创建Flask应用
app = Flask(__name__)
DOC_DIR = os.path.abspath("data")

@app.route("/docs/<path:filename>")
def serve_doc(filename):
    """提供文档访问服务"""
    try:
        return send_from_directory(DOC_DIR, filename, mimetype='text/plain; charset=utf-8')
    except Exception as e:
        print(f"文件访问错误: {e}")
        abort(404)

@app.route("/")
def index():
    """显示可用文档列表"""
    try:
        files = []
        for root, dirs, filenames in os.walk(DOC_DIR):
            for filename in filenames:
                if filename.endswith('.txt'):
                    rel_path = os.path.relpath(os.path.join(root, filename), DOC_DIR)
                    files.append(rel_path.replace('\\', '/'))
        
        html = "<h1>本地文档服务</h1><ul>"
        for file in files:
            html += f'<li><a href="/docs/{file}">{file}</a></li>'
        html += "</ul>"
        return html
    except Exception as e:
        return f"错误: {e}"

def get_local_ip():
    """获取本机IP地址"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

def start_doc_server():
    """启动文档服务器"""
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

# 在后台启动Flask服务器
print("🌐 启动本地文档Web服务...")
server_thread = threading.Thread(target=start_doc_server, daemon=True)
server_thread.start()

# 等待服务器启动
time.sleep(2)

local_ip = get_local_ip()
print(f"✅ 文档服务器已启动")
print(f"📍 本地访问: http://localhost:5000")
print(f"📍 网络访问: http://{local_ip}:5000")
print(f"📁 文档目录: {DOC_DIR}")
print(f"📋 文档列表: http://{local_ip}:5000")

🌐 启动本地文档Web服务...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.10.10.147:5000
Press CTRL+C to quit


✅ 文档服务器已启动
📍 本地访问: http://localhost:5000
📍 网络访问: http://10.10.10.147:5000
📁 文档目录: C:\learn\7-22\llm-developing-mock-interview\data
📋 文档列表: http://10.10.10.147:5000


In [15]:
# 加载文档
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("📖 通过Web服务加载文档...")

# 选择加载方式
load_type = "local"  # 可选: "local" 或 "wiki"

if load_type == "local":
    # 使用本地文档服务
    local_ip = get_local_ip()
    # 修正：使用正确的docs路径
    doc_url = f"http://{local_ip}:5000/docs/绍宋.txt"  # 修复路径
    print(f"🔗 加载本地文档: {doc_url}")
    
    try:
        loader = WebBaseLoader(doc_url)
        docs = loader.load()
        print(f"✅ 本地文档服务加载成功")
    except Exception as e:
        print(f"❌ 本地文档加载失败: {e}")
        print("💡 提示: 确保已运行步骤1C启动文档服务器")
        print("💡 提示: 确保data/cv.txt文件存在")
        print("💡 提示: 访问 http://localhost:5000 查看可用文档")
        docs = []
        
elif load_type == "wiki":
    # 使用维基百科
    doc_url = "https://zh.wikipedia.org/wiki/人工智能"
    print(f"🔗 加载维基百科: {doc_url}")
    loader = WebBaseLoader(doc_url)
    docs = loader.load()
    print(f"✅ 维基百科加载成功")

if docs:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    documents = text_splitter.split_documents(docs)
    print(f"📊 文档分割完成，共{len(documents)}个片段")
    print(f"📝 第一个片段预览: {documents[0].page_content[:100]}...")
else:
    print("⚠️ 没有加载到文档，使用空文档列表")
    documents = []

📖 通过Web服务加载文档...
🔗 加载本地文档: http://10.10.10.147:5000/docs/绍宋.txt
✅ 本地文档服务加载成功
📊 文档分割完成，共143个片段
📝 第一个片段预览: 绍宋
作者：榴弹怕水

内容简介：
　　绍者，一曰继；二曰导。
　　公元1127年，北宋灭亡。旋即，皇九子赵构在万众期待中于商丘登基，继承宋统，改元建炎。
　　然而，三个月内，李纲罢相，陈...


# 💡 本地文档Web服务使用说明

## 🎯 三种文档加载方式对比

1. **本地文件加载** (步骤1A) - 直接读取本地文件
2. **本地Web服务** (步骤1C+1D) - 通过HTTP访问本地文档
3. **网络文档** (步骤1D) - 加载互联网文档

## 🌐 本地Web服务优势

- ✅ 统一的WebBaseLoader接口
- ✅ 支持网络访问（其他设备可访问）
- ✅ 便于调试和查看文档
- ✅ 可以轻松切换不同文档

## 🔧 使用方法

1. 运行步骤1C启动文档服务器
2. 在步骤1D中修改`load_type`和`doc_url`
3. 访问 http://本机IP:5000 查看可用文档列表

In [17]:

from langchain_huggingface import HuggingFaceEmbeddings
# 步骤2: 创建向量搜索
print("🧠 创建向量搜索系统...")

# 使用BGE嵌入模型
model_name = os.getenv('BGE_MODEL_PATH', 'BAAI/bge-small-zh-v1.5')
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# 从配置文件读取Qdrant设置
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant

qdrant_host = os.getenv('QDRANT_HOST', 'localhost')
qdrant_port = int(os.getenv('QDRANT_PORT', 6333))

# 创建Qdrant向量数据库
vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    collection_name="绍宋test",
    host=qdrant_host,
    port=qdrant_port
)

retriever = vector_store.as_retriever()
print("✅ Qdrant向量搜索系统创建完成")

🧠 创建向量搜索系统...
✅ Qdrant向量搜索系统创建完成


In [18]:
# 步骤3: 配置DeepSeek模型
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="deepseek-chat",
    temperature=0,
    openai_api_key=os.getenv("DEEPSEEK_API_KEY"),
    openai_api_base="https://api.deepseek.com"
)

print("🧠 DeepSeek模型配置完成")

🧠 DeepSeek模型配置完成


In [19]:
# 步骤4: 创建搜索工具
from langchain.tools.retriever import create_retriever_tool
search_tool = create_retriever_tool(
    retriever,
    "shaosong_search",
    "搜索绍宋前段内容相关信息"
)

tools = [search_tool]
print("🔧 搜索工具创建完成")

🔧 搜索工具创建完成


In [ ]:
# 步骤5: 获取提示模板
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
print("📋 提示模板加载完成")

In [38]:

# 步骤6: 创建Agent
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

print("🎉 Agent创建成功！")

🎉 Agent创建成功！


In [ ]:
# 测试1: 简单对话
print("测试1: 基础对话")
result = agent_executor.invoke({"input": "你好，请介绍一下自己"})
print(f"回答: {result['output']}")

In [39]:
# 测试2: 知识搜索
print("测试2: 搜索功能")
result = agent_executor.invoke({"input": "请搜索本地知识库，告诉我赵玖在尧山和谁对射？本地知识库没有找到就回答不知道"})
print(f"回答: {result['output']}")

测试2: 搜索功能
回答: 根据本地知识库的检索，没有找到关于赵玖在尧山与谁对射的相关信息。因此，答案是：不知道。


# 🧠 第2课：给Agent添加记忆

**目标**: 让Agent记住对话历史，实现真正的对话体验

**技术要点**: DeepSeek支持多轮对话，记忆管理

In [27]:

print("=" * 60)
print("🔍 测试2: 专业问题（需要搜索工具）")
print("=" * 60)

result1 = agent_executor.invoke({"input": "请搜索本地知识库，告诉我赵玖穿越后，在明道宫遇到的主要困境是什么？"})
print(f"\n✨ Agent回答: {result['output']}")
print("\n💡 观察上面的过程：")
print("   1. Agent分析问题")
print("   2. 决定需要搜索本地知识库")
print("   3. 调用搜索工具")
print("   4. 基于搜索结果生成答案")
print(f"第1轮对话: {result1['output']}")

# 第二次对话（手动传入历史）
result2 = agent_executor.invoke({
    "input": "明道宫谁遇到了困难",
    "chat_history": [
        HumanMessage(content="请搜索本地知识库，告诉我赵玖穿越后，在明道宫遇到的主要困境是什么？"),
        AIMessage(content=result1['output'])
    ]
})
print(f"第2轮对话: {result2['output']}")

print("\n💡 手动方式需要自己维护历史记录列表")

🔍 测试2: 专业问题（需要搜索工具）


> Entering new AgentExecutor chain...
根据本地知识库中《绍宋》的相关内容，赵玖（宋高宗赵构穿越后的身份）在明道宫遭遇的主要困境包括：

1. **政治孤立**  
   - 朝中主和派（如汪伯彦、黄潜善等）势力庞大，极力阻挠赵玖的北伐主张，甚至暗中策划迁都避战，削弱皇权权威。
   - 部分将领和官员对年轻皇帝的能力持怀疑态度，导致政令执行受阻。

2. **军事危机**  
   - 金军持续南侵，南宋防线脆弱，而明道宫作为临时驻跸地缺乏战略纵深，随时面临敌军威胁。
   - 军队系统腐败低效，需赵玖亲自整顿（如通过“苗刘之变”后的清洗重建中枢权威）。

3. **经济困境**  
   - 朝廷财政拮据，难以支撑北伐筹备，需解决粮饷筹措和后勤保障问题。

4. **个人身份挑战**  
   - 作为穿越者，赵玖需快速适应皇帝身份并平衡现代思维与古代政治现实的冲突，同时避免暴露身份异常。

**关键情节提示**：赵玖通过在明道宫“称病”期间秘密接见岳飞、韩世忠等主战派将领，并利用“明道宫对质”等事件逐步瓦解主和派，最终确立抗金路线。这一阶段的困境成为他整肃朝纲、启动“绍宋”新政的重要转折点。

> Finished chain.

✨ Agent回答: 根据本地知识库中《绍宋》的相关内容，赵玖（宋高宗赵构穿越后的身份）在明道宫遭遇的主要困境包括：

1. **政治孤立与信任危机**  
   - 赵玖以"神佑重生"的名义在明道宫即位，但朝廷旧臣（如吕好问、汪伯彦等）对其突然转变的强硬抗金态度充满怀疑，认为他可能是"借尸还魂"的妖孽，暗中阻挠其政令。
   - 以苗傅、刘正彦为首的潜逃武将集团试图发动兵变，质疑其统治合法性。

2. **军事上的绝境**  
   - 金军主力（完颜兀术部）已突破长江防线，距离明道宫仅百里之遥，而南宋残存的御营军队士气崩溃，逃兵现象严重。
   - 核心将领韩世忠重伤未愈，岳飞尚未崛起，赵玖缺乏可直接掌控的军事力量。

3. **经济与后勤崩溃**  
   - 朝廷仓皇南渡时丢失大部分府库积蓄，明道宫周边地区因战乱饥荒，无法维持基本军队补给。
   - 文官系统仍坚持"议和纳贡"的传统思路，拒绝配合赵玖的战争动员。

4. **历史认知的冲突** 

In [40]:
# 🤖 方法2: 自动记忆管理（推荐使用）
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

print("🚀 创建自动记忆Agent")

# 创建记忆存储器
memory = ChatMessageHistory()

# 给Agent添加自动记忆功能
agent_with_memory = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: memory,  # 记忆获取函数
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("✅ 自动记忆Agent创建成功！")
print("🎯 现在Agent会自动记住所有对话内容")

🚀 创建自动记忆Agent
✅ 自动记忆Agent创建成功！
🎯 现在Agent会自动记住所有对话内容


In [41]:
# 🧪 测试自动记忆功能
session_config = {"configurable": {"session_id": "demo"}}

print("🧪 测试自动记忆功能")
print("=" * 50)

result0 = agent_with_memory.invoke(
    {"input": "deepseek的回答太复杂，杂糅很多东西，请你精简"},
    config=session_config
)
print(f"第1轮: {result0['output']}")
# 对话1: 介绍自己
result1 = agent_with_memory.invoke(
    {"input": "我是小明，我正在读绍宋这本书"},
    config=session_config
)
print(f"第1轮: {result1['output']}")

# 对话2: 测试记忆
result2 = agent_with_memory.invoke(
    {"input": "我是谁，我在干什么"},
    config=session_config
)
print(f"第2轮: {result2['output']}")

# 对话3: 结合记忆和搜索
result3 = agent_with_memory.invoke(
    {"input": "请搜索本地知识库，告诉我有没有曲端这个人物，如果本地知识库没有找到，可以回答不知道"},
    config=session_config
)
print(f"第3轮: {result3['output']}")


🧪 测试自动记忆功能
第1轮: 好的！我会用最简洁的方式回答，避免冗余信息。  

**如何精简回答？**  
1. **直接给出核心答案**，省略背景和解释（除非必要）。  
2. **分点或短句**，避免长段落。  
3. **用简单词汇**，减少术语。  

**示例对比**  
❌ 复杂版：  
“根据多方数据分析，考虑到用户可能的需求差异，建议采取分阶段策略，优先优化关键模块，同时兼顾长期可扩展性……”  

✅ 精简版：  
“建议先优化核心功能，后续再扩展。”  

需要我精简什么内容？可以具体告诉我，我会立刻调整！
第1轮: 好的，小明！  

**简短回答：**  
《绍宋》是历史穿越小说，主角赵玖（宋高宗）逆天改命，抗金救国。主线紧凑，战争热血，权谋精彩，配角鲜活。作者文笔好，考据细，但后期节奏略慢。  

**关键词：** 南宋穿越、抗金、权谋、战争、考据党  

需要讨论具体情节或角色吗？可以随时问我！
第2轮: **小明**，你正在读历史穿越小说《绍宋》，主角赵玖（宋高宗）逆天改命抗金。  

需要聊剧情、角色，或帮你找书评/分析吗？ 😊  

（若想切换话题，直接告诉我就行！）
第3轮: 根据《绍宋》原著内容：  

**曲端**是书中重要角色之一，南宋初年西军将领（历史上真实存在），性格桀骜但善战。小说中他前期与主角赵玖有激烈冲突，后期成为抗金核心力量，尤其“鄜延军”情节高燃。  

需要具体情节或分析吗？  

（若本地知识库指其他资料库，请明确说明，我会重新检索！）


# 🛠️ 第3课：创建自定义Agent

**目标**: 从零开始构建专属Agent，掌握核心原理

**技术栈**: DeepSeek + 自定义工具 + 智能提示

In [42]:
# 🔧 步骤1: 创建自定义工具
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """计算单词的字母数量。对于需要知道单词长度的问题很有用。"""
    return len(word)

@tool
def multiply_numbers(a: int, b: int) -> int:
    """计算两个数字的乘积。用于数学乘法运算。"""
    return a * b

@tool
def count_characters(text: str) -> dict:
    """统计文本中的字符数量，返回详细信息。"""
    return {
        "total_chars": len(text),
        "letters_only": sum(c.isalpha() for c in text),
        "numbers_only": sum(c.isdigit() for c in text),
        "spaces": text.count(' ')
    }

# 测试工具
print("🧪 测试自定义工具:")
print(f"get_word_length('Python'): {get_word_length.invoke('Python')}")
print(f"multiply_numbers(6, 7): {multiply_numbers.invoke({'a': 6, 'b': 7})}")
print(f"count_characters('Hello 123'): {count_characters.invoke('Hello 123')}")

custom_tools = [get_word_length, multiply_numbers, count_characters]
print(f"\n✅ 创建了 {len(custom_tools)} 个专业工具！")

🧪 测试自定义工具:
get_word_length('Python'): 6
multiply_numbers(6, 7): 42
count_characters('Hello 123'): {'total_chars': 9, 'letters_only': 5, 'numbers_only': 3, 'spaces': 1}

✅ 创建了 3 个专业工具！


In [43]:
# 📝 步骤2: 设计专属提示模板
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

custom_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "你是一个文本分析和数学计算专家。\n"
        "- 当用户问关于单词长度时，使用 get_word_length 工具\n"
        "- 当用户需要乘法计算时，使用 multiply_numbers 工具\n"
        "- 当用户需要详细的文本统计时，使用 count_characters 工具\n"
        "你总是优先使用工具，而不是猜测答案。"
    ),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

print("📋 专属提示模板创建完成")
print("💡 这个Agent专门擅长文本分析和数学计算")

📋 专属提示模板创建完成
💡 这个Agent专门擅长文本分析和数学计算


In [44]:
# 🔗 步骤3: 组装自定义Agent
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

# 将工具绑定到LLM
llm_with_custom_tools = llm.bind_tools(custom_tools)

print(f"🔗 工具绑定完成，共绑定 {len(custom_tools)} 个工具")

# 手动组装Agent（深入理解原理）
custom_agent = (
    {
        "input": lambda x: x["input"],  # 用户输入
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]  # 中间步骤（工具调用历史）
        ),
    }
    | custom_prompt              # 自定义提示
    | llm_with_custom_tools      # 绑定工具的LLM
    | OpenAIToolsAgentOutputParser()  # 输出解析器
)

# 创建执行器
custom_agent_executor = AgentExecutor(
    agent=custom_agent,
    tools=custom_tools,
    verbose=True
)

print("🎉 自定义Agent构建成功！")
print("💪 这是你从零开始创建的专属AI助手")

🔗 工具绑定完成，共绑定 3 个工具
🎉 自定义Agent构建成功！
💪 这是你从零开始创建的专属AI助手


In [45]:
# 🧪 测试自定义Agent的专业技能

print("=" * 60)
print("🧪 测试1: 单词长度分析")
print("=" * 60)

result = custom_agent_executor.invoke({
    "input": "How many letters are in the word 'LangChain'?"
})
print(f"✨ 结果: {result['output']}")

🧪 测试1: 单词长度分析


> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'LangChain'}`
responded: I'll use the word length tool to count the letters in 'LangChain'.

9The word 'LangChain' contains 9 letters.

> Finished chain.
✨ 结果: The word 'LangChain' contains 9 letters.


In [46]:
print("=" * 60)
print("🧪 测试2: 数学计算")
print("=" * 60)

result = custom_agent_executor.invoke({
    "input": "What is 23 multiplied by 17?"
})
print(f"✨ 结果: {result['output']}")

🧪 测试2: 数学计算


> Entering new AgentExecutor chain...

Invoking: `multiply_numbers` with `{'a': 23, 'b': 17}`
responded: I'll calculate 23 multiplied by 17 for you.

39123 multiplied by 17 equals 391.

> Finished chain.
✨ 结果: 23 multiplied by 17 equals 391.


In [47]:
print("=" * 60)
print("🧪 测试3: 复合任务（展示Agent的智能）")
print("=" * 60)

result = custom_agent_executor.invoke({
    "input": "The word 'OpenAI' has how many letters? Then multiply that number by 8."
})
print(f"✨ 结果: {result['output']}")

print("\n🎯 观察Agent的智能行为：")
print("   1. 理解复合任务")
print("   2. 分解为两个步骤")
print("   3. 依次调用不同工具")
print("   4. 整合结果给出最终答案")

🧪 测试3: 复合任务（展示Agent的智能）


> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'OpenAI'}`
responded: I'll help you find the length of the word 'OpenAI' and then multiply it by 8.

6
Invoking: `multiply_numbers` with `{'a': 6, 'b': 8}`
responded: Now I'll multiply that number (6) by 8:

48The word 'OpenAI' has 6 letters. When you multiply 6 by 8, you get 48.

> Finished chain.
✨ 结果: The word 'OpenAI' has 6 letters. When you multiply 6 by 8, you get 48.

🎯 观察Agent的智能行为：
   1. 理解复合任务
   2. 分解为两个步骤
   3. 依次调用不同工具
   4. 整合结果给出最终答案


In [48]:
print("=" * 60)
print("🧪 测试4: 文本详细分析")
print("=" * 60)

result = custom_agent_executor.invoke({
    "input": "Please analyze this text: 'Hello World 123!'"
})
print(f"✨ 结果: {result['output']}")

🧪 测试4: 文本详细分析


> Entering new AgentExecutor chain...

Invoking: `count_characters` with `{'text': 'Hello World 123!'}`
responded: I'll analyze the text 'Hello World 123!' for you using the character counting tool.

{'total_chars': 16, 'letters_only': 10, 'numbers_only': 3, 'spaces': 2}Here's the detailed analysis of the text 'Hello World 123!':

- **Total characters**: 16 characters
- **Letters only**: 10 letters
- **Numbers only**: 3 digits
- **Spaces**: 2 spaces

The text also contains 1 punctuation mark (the exclamation point '!'), which accounts for the remaining character in the total count.

> Finished chain.
✨ 结果: Here's the detailed analysis of the text 'Hello World 123!':

- **Total characters**: 16 characters
- **Letters only**: 10 letters
- **Numbers only**: 3 digits
- **Spaces**: 2 spaces

The text also contains 1 punctuation mark (the exclamation point '!'), which accounts for the remaining character in the total count.


# 📅 第4课：实际应用案例

**目标**: 创建一个实用的日期助手Agent

**应用场景**: 日期计算、星期查询、年龄统计等实际需求

In [49]:
# 📅 创建实用的日期工具套件
from datetime import datetime, date, timedelta
import calendar

@tool
def get_current_date() -> str:
    """获取今天的日期，格式为YYYY-MM-DD。"""
    return str(date.today())

@tool
def get_day_of_week(date_str: str) -> str:
    """获取指定日期是星期几。日期格式：YYYY-MM-DD。"""
    try:
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")
        day_names = ['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日']
        return day_names[date_obj.weekday()]
    except ValueError:
        return "日期格式错误，请使用YYYY-MM-DD格式"

@tool
def calculate_days_between(start_date: str, end_date: str) -> int:
    """计算两个日期之间的天数差。日期格式：YYYY-MM-DD。"""
    try:
        start = datetime.strptime(start_date, "%Y-%m-%d").date()
        end = datetime.strptime(end_date, "%Y-%m-%d").date()
        return abs((end - start).days)
    except ValueError:
        return -1

@tool
def calculate_age_in_days(birth_date: str) -> dict:
    """计算从出生日期到今天的详细信息。出生日期格式：YYYY-MM-DD。"""
    try:
        birth = datetime.strptime(birth_date, "%Y-%m-%d").date()
        today = date.today()
        days_lived = (today - birth).days
        years = days_lived // 365
        return {
            "days_lived": days_lived,
            "approximate_years": years,
            "approximate_months": days_lived // 30,
            "birth_day_of_week": get_day_of_week.invoke(birth_date)
        }
    except ValueError:
        return {"error": "日期格式错误"}

date_tools = [get_current_date, get_day_of_week, calculate_days_between, calculate_age_in_days]

# 测试日期工具
print("📅 日期工具测试：")
print(f"今天是: {get_current_date.invoke({})}")
print(f"2024-01-01是: {get_day_of_week.invoke('2024-01-01')}")
print(f"从2024-01-01到2024-12-31共: {calculate_days_between.invoke({'start_date': '2024-01-01', 'end_date': '2024-12-31'})} 天")

print(f"\n✅ 创建了 {len(date_tools)} 个实用的日期工具！")

📅 日期工具测试：
今天是: 2025-07-24
2024-01-01是: 星期一
从2024-01-01到2024-12-31共: 365 天

✅ 创建了 4 个实用的日期工具！


In [50]:
# 👨‍💼 创建专业的日期助手Agent
date_assistant_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "你是一个专业的日期计算助手。你可以：\n"
        "1. 获取当前日期\n"
        "2. 查询任意日期是星期几\n"
        "3. 计算两个日期间的天数差\n"
        "4. 计算年龄和生活天数\n\n"
        "请始终使用相应的工具来获取准确信息，不要猜测日期。"
        "所有日期请使用YYYY-MM-DD格式。"
    ),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 组装日期助手
llm_with_date_tools = llm.bind_tools(date_tools)

date_assistant = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | date_assistant_prompt
    | llm_with_date_tools
    | OpenAIToolsAgentOutputParser()
)

date_assistant_executor = AgentExecutor(
    agent=date_assistant,
    tools=date_tools,
    verbose=True
)

print("📅 专业日期助手创建完成！")
print("💼 这个助手可以处理各种日期相关的实际问题")

📅 专业日期助手创建完成！
💼 这个助手可以处理各种日期相关的实际问题


In [51]:
# 🧪 测试日期助手的实用功能

print("=" * 60)
print("📅 测试1: 获取当前日期")
print("=" * 60)

result = date_assistant_executor.invoke({
    "input": "今天是几号？"
})
print(f"✨ 结果: {result['output']}")

📅 测试1: 获取当前日期


> Entering new AgentExecutor chain...

Invoking: `get_current_date` with `{}`


2025-07-24今天是2025年7月24日。

> Finished chain.
✨ 结果: 今天是2025年7月24日。


In [52]:
print("=" * 60)
print("📅 测试2: 查询星期几")
print("=" * 60)

result = date_assistant_executor.invoke({
    "input": "2024年圣诞节（2024-12-25）是星期几？"
})
print(f"✨ 结果: {result['output']}")

📅 测试2: 查询星期几


> Entering new AgentExecutor chain...

Invoking: `get_day_of_week` with `{'date_str': '2024-12-25'}`
responded: 我来帮您查询2024年圣诞节是星期几。

星期三2024年圣诞节（2024-12-25）是**星期三**。

> Finished chain.
✨ 结果: 2024年圣诞节（2024-12-25）是**星期三**。


In [53]:
print("=" * 60)
print("📅 测试3: 计算时间间隔")
print("=" * 60)

result = date_assistant_executor.invoke({
    "input": "从2024年1月1日到2024年12月31日一共有多少天？"
})
print(f"✨ 结果: {result['output']}")

📅 测试3: 计算时间间隔


> Entering new AgentExecutor chain...

Invoking: `calculate_days_between` with `{'start_date': '2024-01-01', 'end_date': '2024-12-31'}`
responded: 我来帮您计算从2024年1月1日到2024年12月31日一共有多少天。

365从2024年1月1日到2024年12月31日一共有**365天**。

需要注意的是，虽然2024年是闰年（有366天），但从1月1日到12月31日的天数差计算结果是365天，这是因为计算的是两个日期之间的间隔天数，不包括起始日期本身。如果要包括起始和结束日期，那么2024年全年就是366天。

> Finished chain.
✨ 结果: 从2024年1月1日到2024年12月31日一共有**365天**。

需要注意的是，虽然2024年是闰年（有366天），但从1月1日到12月31日的天数差计算结果是365天，这是因为计算的是两个日期之间的间隔天数，不包括起始日期本身。如果要包括起始和结束日期，那么2024年全年就是366天。


In [56]:
print("=" * 60)
print("📅 测试4: 年龄计算（实用功能）")
print("=" * 60)

result = date_assistant_executor.invoke({
    "input": "如果我2003年2月18日出生，我现在活了多少天？大概多少岁？"
})
print(f"✨ 结果: {result['output']}")

result = date_assistant_executor.invoke({
    "input": "请搜索本地知识库，介绍一下绍宋这本书？"
})
print(f"✨ 结果: {result['output']}")

print("\n🎯 实际应用价值：")
print("   ✅ 可以集成到生日提醒系统")
print("   ✅ 可以用于项目时间管理")
print("   ✅ 可以帮助HR计算工作日")
print("   ✅ 可以用于历史事件查询")

📅 测试4: 年龄计算（实用功能）


> Entering new AgentExecutor chain...

Invoking: `calculate_age_in_days` with `{'birth_date': '2003-02-18'}`
responded: 我来帮你计算一下从2003年2月18日出生到现在的详细信息。

{'days_lived': 8192, 'approximate_years': 22, 'approximate_months': 273, 'birth_day_of_week': '星期二'}根据计算结果，如果你是2003年2月18日出生的：

**🎂 年龄信息：**
- **大概年龄**：22岁
- **生活天数**：8,192天
- **生活月数**：约273个月

**📅 其他信息：**
- 你出生的那天是星期二
- 从出生到现在已经度过了超过8千天的时光

真是一个有意义的里程碑！8,192天的人生经历，相当丰富呢！

> Finished chain.
✨ 结果: 根据计算结果，如果你是2003年2月18日出生的：

**🎂 年龄信息：**
- **大概年龄**：22岁
- **生活天数**：8,192天
- **生活月数**：约273个月

**📅 其他信息：**
- 你出生的那天是星期二
- 从出生到现在已经度过了超过8千天的时光

真是一个有意义的里程碑！8,192天的人生经历，相当丰富呢！


> Entering new AgentExecutor chain...
我很抱歉，但我是一个专业的日期计算助手，我的功能专门用于处理日期相关的计算，包括：

1. 获取当前日期
2. 查询任意日期是星期几  
3. 计算两个日期间的天数差
4. 计算年龄和生活天数

我没有搜索本地知识库或介绍书籍的功能。如果您需要了解《绍宋》这本书的信息，建议您：

- 咨询图书管理员或书店工作人员
- 在图书网站或数据库中搜索
- 使用专门的图书搜索工具

如果您有任何日期计算方面的问题，我很乐意为您提供帮助！比如计算某个历史事件的日期间隔、查询特定日期是星期几等。

> Finished chain.
✨ 结果: 我很抱歉，但我是一个专业的日期计算助手，我的功能专门用于处理日期相关的计算，包括：

1. 获取当前日期
2

# 🎓 恭喜！你已经掌握了基于DeepSeek的LangChain Agent

## 🏆 你的学习成果

### 💪 核心技能
- ✅ **Agent基础架构**: 理解DeepSeek + 工具 + 提示的核心模式
- ✅ **快速上手**: 能够使用国产模型解决实际问题
- ✅ **记忆管理**: 掌握多轮对话，创建智能交互体验
- ✅ **自定义开发**: 从零构建专属Agent，包括工具设计和提示工程
- ✅ **实际应用**: 创建实用的Agent解决真实业务问题

### 🛠️ 实践项目
1. **📚 文档检索Agent** - 智能问答系统的基础
2. **🧮 计算助手Agent** - 文本分析和数学计算专家
3. **📅 日期助手Agent** - 实用的日期计算工具

### 💰 成本优化方案
- ✅ DeepSeek模型：成本仅为GPT-4的1/50
- ✅ BGE本地嵌入：完全免费，无API调用费用
- ✅ FAISS本地存储：无额外云存储费用
- ✅ **总体成本比OpenAI方案降低95%以上！**

## 🚀 下一步进阶方向

### 🎯 推荐学习路径
1. **多Agent协作** - 让多个DeepSeek Agent协同工作
2. **RAG系统** - 结合本地知识库的高级问答
3. **流式响应** - 实时交互体验优化
4. **生产部署** - 将Agent部署到实际业务环境

### 🇨🇳 国产化技术栈
- [DeepSeek官方文档](https://platform.deepseek.com/docs)
- [BGE嵌入模型](https://huggingface.co/BAAI/bge-large-zh-v1.5)
- [Qdrant向量数据库](https://qdrant.tech/)（可选升级方案）

## 💡 最后的建议

1. **拥抱国产**: DeepSeek在中文理解和推理能力上表现优秀
2. **成本意识**: 合理的技术选型能显著降低运营成本
3. **持续实践**: Agent开发需要大量实战，建议多做项目练习
4. **社区参与**: 关注国产AI模型社区，获取最新技术动态

**🎉 你现在已经掌握了基于国产模型的AI Agent开发技能，快去创造属于中国的智能应用吧！**